In [61]:
import os
import pandas as pd

rel_path = "./data/uveitis_data.xlsx"

def read_data(rel_path, **kwargs):
    filename, file_extension = os.path.splitext(rel_path)
    print(os.path.basename(rel_path) + " will be read")
    if file_extension == ".xlsx":
        df = pd.read_excel(rel_path, **kwargs)
        print("reading filetype "+file_extension)
    elif file_extension == ".csv":
        df = pd.read_csv(rel_path, **kwargs)
        print("reading filetype "+file_extension)
    elif file_extension == ".json":
        df = pd.read_json(rel_path, **kwargs)
        print("reading filetype "+file_extension)
    elif file_extension == ".html":
        df = pd.read_html(rel_path, **kwargs)
        print("reading filetype "+file_extension)
    else:
        print("Filetype not supported by function")
        df = 0

    if df.size != 0:
        print("successfully created table with ", df.size, "values and loaded as df")
        print("the table is ",df.shape[1], "wide and ",df.shape[0],"long")

read_data(rel_path)

D:\Drive\FHNW\zaRepos\fhnw_ds_fs2021_medical_challenge
uveitis_data.xlsx will be read
reading filetype .xlsx
successfully created table with  126850 values and loaded as df
the table is  118 wide and  1075 long
